In [ ]:
import pickle

# These are from Data preprocessing notebook and pickles 

non_validation_data = pd.read_pickle(r'non_validation_data.p')
validation_data = pd.read_pickle(r'validation_data.p')

# Feature Engineering Functions

## Feature - Total Buys and Ratios

In [ ]:
# total buy features 

def generate_total_buy_7(df):
    total_buy_7 = df[df.order_number_reverse<=6].groupby(['user_id', 'product_id']).size().reset_index()
    total_buy_7.columns = ['user_id','product_id','total_buy_7']
    total_buy_7['total_buy_ratio_7'] = total_buy_7['total_buy_7']/7  
    return total_buy_7

def generate_total_buy_6(df):
    total_buy_6 = df[df.order_number_reverse<=5].groupby(['user_id', 'product_id']).size().reset_index()
    total_buy_6.columns = ['user_id','product_id','total_buy_6']
    total_buy_6['total_buy_ratio_6'] = total_buy_6['total_buy_6']/6  
    return total_buy_6

def generate_total_buy_5(df):
    total_buy_5 = df[df.order_number_reverse<=4].groupby(['user_id', 'product_id']).size().reset_index()
    total_buy_5.columns = ['user_id','product_id','total_buy_5']
    total_buy_5['total_buy_ratio_5'] = total_buy_5['total_buy_5']/5  
    return total_buy_5

def generate_total_buy_4(df):
    total_buy_4 = df[df.order_number_reverse<=3].groupby(['user_id', 'product_id']).size().reset_index()
    total_buy_4.columns = ['user_id','product_id','total_buy_4']
    total_buy_4['total_buy_ratio_4'] = total_buy_4['total_buy_4']/4
    return total_buy_4

def generate_total_buy_3(df):
    total_buy_3 = df[df.order_number_reverse<=2].groupby(['user_id', 'product_id']).size().reset_index()
    total_buy_3.columns = ['user_id','product_id','total_buy_3']
    total_buy_3['total_buy_ratio_3'] = total_buy_3['total_buy_3']/3    
    return total_buy_3

def generate_total_buy_all(df):
    total_buy_all = df.groupby(['user_id', 'product_id']).size().reset_index() 
    total_buy_all.columns = ['user_id','product_id','total_buy_all']
    return total_buy_all

##  Feature - Repurchase Period

In [ ]:
#longest days without repurchase category

def generate_longest_period_wo_repurchase(df):
    
    longest_period = df.groupby(['user_id', 'product_id'])['days_since_prior_order'].max().reset_index()
    longest_period.rename(columns={'days_since_prior_order':'longest_period_without_repurchasing_in_all_orders'}, inplace=True)

    return longest_period

def generate_longest_period_wo_repurchase_3(df):
    
    longest_period_3 = df[df.order_number_reverse<=2].groupby(['user_id', 'product_id'])['days_since_prior_order'].max().reset_index()
    longest_period_3.rename(columns={'days_since_prior_order':'longest_period_without_repurchasing_in_recent_3_orders'}, inplace=True)

    return longest_period_3
    
def generate_longest_period_wo_repurchase_5(df):
    
    longest_period_5 = df[df.order_number_reverse<=4].groupby(['user_id', 'product_id'])['days_since_prior_order'].max().reset_index()
    longest_period_5.rename(columns={'days_since_prior_order':'longest_period_without_repurchasing_in_recent_5_orders'}, inplace=True)

    return longest_period_5

def median_days_wo_repurchase_n5(df):
    median_without_repurchasing_5 = df[df.order_number_reverse<=4].groupby(['user_id', 'product_id'])['days_since_prior_order'].median().reset_index()
    median_without_repurchasing_5.rename(columns={'days_since_prior_order':'median_without_repurchasing_in_recent_5_orders'}, inplace=True)
    
    return median_without_repurchasing_5

## Feature - Chance

In [ ]:
def generate_chance_and_ratio(df):
    
    cnt = df.groupby(['user_id', 'product_id']).size()
    cnt.name = 'cnt'
    cnt = cnt.reset_index()
    
    # chance <-- itself is a feature
    
    # The latest order
    user_onb_max = df.groupby('user_id').order_number.max().reset_index() 
    user_onb_max.columns = ['user_id', 'onb_max']
    
    # The first order containing the pdt
    user_item_min = df.groupby(['user_id', 'product_id']).order_number.min().reset_index() 
    user_item_min.columns = ['user_id', 'product_id', 'onb_min']
    
    chance = pd.merge(user_item_min, user_onb_max, on='user_id', how='left')
    
    # Since its first order of this pdt, how many orders have been made
    chance['chance'] = chance.onb_max - chance.onb_min +1   
    
    # cnt-> how many times did the user bought the pdt
    final = pd.merge(cnt, chance, on=['user_id', 'product_id'], how='left')  
    
    # calculate that ratio
    final['order_ratio_bychance'] = final.cnt / final.chance    
    final =final.drop(['onb_max','onb_min','cnt'],axis=1)
    return final


def generate_chance_and_ratio_recent_5(df1):
    
    df = df1[df1.order_number_reverse<=4]
    
    cnt = df.groupby(['user_id', 'product_id']).size()
    cnt.name = 'cnt'
    cnt = cnt.reset_index()
    
    # The latest order
    user_onb_max = df.groupby('user_id').order_number.max().reset_index() 
    user_onb_max.columns = ['user_id', 'onb_max']
    
    # The first order containing the pdt
    user_item_min = df.groupby(['user_id', 'product_id']).order_number.min().reset_index() 
    user_item_min.columns = ['user_id', 'product_id', 'onb_min']
    
    chance = pd.merge(user_item_min, user_onb_max, on='user_id', how='left')
    
    # Since its first order of this pdt, how many orders have been made, for the recent 5 orders
    chance['chance_n5'] = chance.onb_max - chance.onb_min +1   
    
    # cnt-> how many times did the user bought the pdt
    final = pd.merge(cnt, chance, on=['user_id', 'product_id'], how='left')  
    
    # calculate that ratio
    final['order_ratio_bychance_n5'] = final.cnt / final.chance_n5  
    final =final.drop(['onb_max','onb_min','cnt'],axis=1)
    
    return final

## Feature - Days since first order

In [ ]:
def generate_days_since_first_orders(df):
    df['days_since_first_order'] = df.sort_values('order_number').groupby(['user_id','product_id']).days_since_prior_order.cumsum()
    df = df.groupby(['user_id','product_id'])['days_since_first_order'].max().reset_index(name='days_since_first_order')
    return df 

# Feature Generation

In [ ]:
total_buy = generate_total_buy_all(non_validation_data)
total_buy = pd.merge(left = total_buy,right=generate_total_buy_7(non_validation_data), how='left')
total_buy = pd.merge(left = total_buy,right=generate_total_buy_6(non_validation_data), how='left')
total_buy = pd.merge(left = total_buy,right=generate_total_buy_5(non_validation_data), how='left')
total_buy = pd.merge(left = total_buy,right=generate_total_buy_4(non_validation_data), how='left')
total_buy = pd.merge(left = total_buy,right=generate_total_buy_3(non_validation_data), how='left')



In [ ]:
total_buy = pd.merge(left = total_buy,right=generate_longest_period_wo_repurchase(non_validation_data), how='left')
total_buy = pd.merge(left = total_buy,right=generate_longest_period_wo_repurchase_3(non_validation_data), how='left')
total_buy = pd.merge(left = total_buy,right=generate_longest_period_wo_repurchase_5(non_validation_data), how='left')
total_buy = pd.merge(left = total_buy,right=median_days_wo_repurchase_n5(non_validation_data), how='left')


In [ ]:
total_buy = pd.merge(left = total_buy,right=generate_chance_and_ratio(non_validation_data), how='left')
total_buy = pd.merge(left = total_buy,right=generate_chance_and_ratio_recent_5(non_validation_data), how='left')



In [ ]:
days_since_first_orders = generate_days_since_first_orders(non_validation_data)


In [ ]:
features_all = pd.merge(left = total_buy,right=days_since_first_orders, how='left')
features_all = features_all.fillna(0)


# Label Generation - Reorder in Recent 5

## Creating the Reordered Column for the recent 5 orders

In [ ]:
# Getting the recent 5 orders 
recent_5_orders = non_validation_data[non_validation_data["order_number_reverse"] <= 5][['order_id','product_id','user_id']]


In [ ]:
# get number_of_order_in_recent_5 at the user product level 
recent_5_reordered = recent_5_orders.groupby(['user_id','product_id']).size().reset_index(name='number_of_order_in_recent_5')


In [ ]:
# If number_of_order_in_recent_5 is one, it means that the user only order it once, so reodered is false 
# if number_of_order_in_recent_5 is more than one, it means there is reordering took place 

recent_5_reordered['reordered_recent_5'] = np.where(recent_5_reordered['number_of_order_in_recent_5']==1 , 0, 1)
recent_5_reordered

In [ ]:
# Drop number_of_order_in_recent_5 for recent_5_reordered
recent_5_reordered = recent_5_reordered.drop(['number_of_order_in_recent_5'],axis=1)

In [ ]:
# get all user-product matrix for non-validation dataset 

user_purchased_product_non_validation = non_validation_data.groupby(['user_id','product_id']).size().reset_index(name='no')
user_purchased_product_non_validation = user_purchased_product_non_validation.iloc[:,:-1]

In [ ]:
# Merge user_purchased_product_non_validation with recent_5_reordered


train_with_all_user_purchased_product = pd.merge(user_purchased_product_non_validation, recent_5_reordered,how="left",on=['user_id',"product_id"])

# if it is NaN after merge, it shows that the user never purchase this product in recent 5 orders
train_with_all_user_purchased_product = train_with_all_user_purchased_product.fillna(0)


# Merging Features and Labels for Validation

In [ ]:
to_train_1 = pd.merge(left=features_all, right=train_with_all_user_purchased_product,on=['user_id','product_id'],how='left')
to_train_1.to_pickle('to_train_1.p')


# Prepare test data at Validation  

In [ ]:
# for validation set

# test_data_1
# test data of user-product level matrix 12422758 rows is for orders up till the order before validation dataset inclusive. 

# get all the user-product from history 
test_data_1 = non_validation_data.groupby(['user_id','product_id']).size().reset_index()

# remove the last column
test_data_1 = test_data_1.iloc[:,:-1]

# merge with the lastest order (validation data) to get reordered 
test_data_1 = pd.merge(test_data_1,validation_data[['user_id','product_id','reordered']],on=['user_id','product_id'], how='left')

# NaN means that the user never order the product in the validation order. Hence 0 is given since no reorder
test_data_1 = test_data_1.fillna(0)

test_data_1.to_pickle('to_test_1.p')


# Prepare for training and testing data after validation
Repeat step 2 - 5 on order_master (include both validation and non-validation data) to get training and test data after validation

They are saved to to_train_2.p and to_test_2.p